# Import and install modules

In [ ]:
#!pip install azureml.core
#!pip install azureml.train
#!pip install azureml.widgets

In [ ]:
import os
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

# Compute Environment

Update the cluster to the existing compute name or if there is not an existing compute it will create one for you with the `provisioning_configuration` specificed.

In [ ]:
cluster = 'gandalf'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC12', min_nodes=1, max_nodes=5)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)
    

# Run Experiment

In [ ]:
# Create and run experiment
mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

run = mnist.submit(estimator)

In [ ]:
run

In [ ]:
#The Experiment has now been created in Azure Machine Learning Workspace. To view it go back to the portals and click on the `Expirements` nav menu
RunDetails(run).show()

# Manage Data

In [ ]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = '.', target_path = 'mnist', show_progress = True)

In [ ]:
# run the same was as above
script_params={
    '--data': mnist_data.as_mount()
}

# Create and run experiment
mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

run = mnist.submit(estimator)
RunDetails(run).show()